Here we try a different approach based on the work of Leung, discussed here: https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

The main idea is to train on several season (5 or 6) with a traditional test/train split
Then, in addition to test scores on those training seasons we check the accuracy on the following two seasons 20162017 and 20172018

For this to make sense we use features that do not involve the identity of the teams, ie we do not make dummy variables for each team
We keeo track of the difference between home team and away team for things like expecected goals % measured for each team abd averaged 
for all games up to the game before. We also use the idea of using the difference of home team - away team stats (rather than all stats for home and all stats for away
team which would be twoce as many features; this idea seems to have orginated with  Pischedda in 2013.

For this preliminary try we use similar features to what Leung used. But I will explore 
to see what happens when the features are varried; there are a ton of options to explore in this direction.

Prelimary results: 


1.  Classifier models have much improved accuracy over the results so far for model 2 (Pischedda approach): 
many of the models have accuracy in the 

58-59% range predicting on 2016-17 and 2017-18 (and f1 scores are solid also)

the test performance on the earlier training seasons is even better 

at 60-61%, 


which makes sense since the testing on an entire following season after training  and even two seasons after testing 
is a more stringent test of the models.

2. Three regressors are also checked at the bottom and their perfomance is even better.
The regressor is fit to predict the goal differential home goals- away goals
Then the regression prediction is used to predict home win or loss. This win/loss prediction 
has:


60-61% accuracy 

for random-forrest and Ridge linear regression( alpha = 0.001)

xgboost regressor is only 55% but this is a complex model thaths many hyper-parameters that need
tuning so there is room to improve that model. It may turn out to be the top performer when properly tuned (as it often does).



Note on tuning: The clasifier models are currently tuned somewhat haphazardly by hand 
Comparing with the untuned models we can see that all models are not much improved or not improved
on their test results in the test/train split on seasons <=2015

So the tuning is not having any significant effect in this run.
More systematic tuning using random search and grid search, using the training seasons is in order.





In [5]:

import pandas as pd
import numpy as np

In [6]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score



##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor





##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()

In [7]:
X = pd.read_csv('/Users/joejohns/data_bootcamp/GitHub/nhl_prediction_Aug_2021B/Data/Processed_Data/Approach_3_bulk_Leung_data/data_LJ.csv')
X.shape

(10385, 26)

In [8]:
X['goal_diff_target'] = X['home_goals'] - X['away_goals']


In [9]:
#we will use this for correlations etc at the bottom

data = X.copy()
X.columns

Index(['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'home_win',
       'settled_in', 'CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv',
       'goal_diff_target'],
      dtype='object')

In [10]:
 
X['season'] = X['season'].apply(int)
X['game_id'] = X['game_id'].apply(int)
X['mp_date'] = X['mp_date'].apply(int)

In [11]:
filt_no_early  = (X['mp_date'].apply(lambda x : x% 10**4) < 900) | (1100 < X['mp_date'].apply(lambda x : x% 10**4))

X = X.loc[filt_no_early, : ].copy()  ## keep games < 900 and > 1100
X.shape

(9104, 27)

In [12]:
X['season'].value_counts()

20162017    1060
20172018    1052
20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [13]:
y_cols = ['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'goal_diff_target', 'home_win',
       'settled_in', ]

x_cols = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']
columns_to_scale = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']  ##same

In [14]:


x = np.array(X.loc[(X['season'] <= 20152016), x_cols].copy())
  #features test-train
Y = X.loc[(X['season'] <= 20152016), y_cols].copy()
   #targets
y = np.array(Y['home_win']).reshape(-1,1)


              
x_16 = X.loc[(X['season'] == 20162017), x_cols].copy()  #features test-train
Y_16 = X.loc[(X['season'] == 20162017), y_cols].copy()   #targets
y_16 = np.array(Y_16['home_win']).reshape(-1,1)

x_17 = X.loc[(X['season'] == 20172018), x_cols].copy()  #features test-train
Y_17 = X.loc[(X['season'] == 20172018), y_cols].copy()   #targets
y_17 = np.array(Y_17['home_win']).reshape(-1,1)




In [15]:
X.loc[(X['season'] >= 20162017), :]['season'].value_counts()

20162017    1060
20172018    1052
Name: season, dtype: int64

In [16]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [17]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts().sum()

6992

In [18]:
print(X.shape)
print(x.shape, Y.shape, y.shape)
print(x_16.shape, y_16.shape)
print(x_17.shape, y_17.shape)
print('sum of ', y.shape[0],y_16.shape[0], y_17.shape[0], ' is: ', y.shape[0] + y_16.shape[0]+y_17.shape[0])

(9104, 27)
(6992, 14) (6992, 13) (6992, 1)
(1060, 14) (1060, 1)
(1052, 14) (1052, 1)
sum of  6992 1060 1052  is:  9104


In [19]:
Y['season'].value_counts()

20142015    1046
20112012    1037
20152016    1036
20132014     994
20082009     971
20102011     962
20092010     946
Name: season, dtype: int64

In [20]:
Y_16['season'].value_counts()

20162017    1060
Name: season, dtype: int64

In [21]:
Y_17['season'].value_counts()

20172018    1052
Name: season, dtype: int64

In [22]:


from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()

In [23]:
###NOT tuned 


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=314, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)
##classifier models
lrc2 = RidgeClassifier(alpha =0.001)
gnb2 = GaussianNB()
lgr2 = LogisticRegression(random_state = 0, max_iter = 500)
svc2 = SVC(kernel = 'rbf')

#tree-based classifiers
rfc2 =  RandomForestClassifier( random_state=0)
bc2 = BaggingClassifier()
gbc2 = GradientBoostingClassifier()
xgbc2 = XGBClassifier()
##quick checks 
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
   # f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'TEST: ', acc ) #, f1, 'training : ', acct, f1t)

SVC() TEST:  0.5961401000714797
RidgeClassif TEST:  0.6025732666190136
RandomForest TEST:  0.5654038598999285
GaussianNB() TEST:  0.5954253037884203
LogisticRegr TEST:  0.6025732666190136
BaggingClass TEST:  0.5160829163688349
GradientBoos TEST:  0.6111508220157256
[18:50:26] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie TEST:  0.544674767691208


In [24]:
##TUNED! By hand ... improving test score on on seasons <= 2015

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=314, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)
##classifier models
lrc = RidgeClassifier(alpha =0.2)
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0, C =10, max_iter = 500)
svc = SVC(kernel = 'rbf', C =10)

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=10, random_state=0, n_estimators = 40)
bc = BaggingClassifier(n_estimators  = 60, max_samples = 0.85)
gbc = GradientBoostingClassifier(learning_rate =0.1, n_estimators = 40, max_depth =4 )
xgbc = XGBClassifier(n_estimators= 45, eta=0.05)

##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'TEST: ', acc, f1, 'training : ', acct, f1t)

SVC(C=10) TEST:  0.6018584703359543 0.6944596818431157 training :  0.6184516359735384 0.7056551724137932
RidgeClassif TEST:  0.6047176554681916 0.6983087834151664 training :  0.5678526729840873 0.667948894078857
RandomForest TEST:  0.5868477483917084 0.6685779816513763 training :  0.8342571070981584 0.8634958032690326
GaussianNB() TEST:  0.5954253037884203 0.6310299869621905 training :  0.5571249776506347 0.5914563747319809
LogisticRegr TEST:  0.6040028591851322 0.6972677595628415 training :  0.5683890577507599 0.6679504814305365
BaggingClass TEST:  0.5511079342387419 0.6089663760896638 training :  0.9998212050777758 0.9998372660699756
GradientBoos TEST:  0.6061472480343102 0.690275435637999 training :  0.6660110852851779 0.7351857102353275
[18:51:04] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logisti

/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie TEST:  0.5875625446747677 0.6662810873337188 training :  0.7216163060969069 0.7750975010833455


In [25]:
x_17_sc = std_scal.transform(x_17)
x_test2 = x_17_sc.copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'Tuned model TEST results on 2016-17: ', acc, f1)

SVC(C=10) Tuned model TEST results on 2016-17:  0.6017110266159695 0.7071977638015374
RidgeClassif Tuned model TEST results on 2016-17:  0.6055133079847909 0.7205387205387206
RandomForest Tuned model TEST results on 2016-17:  0.5817490494296578 0.6774193548387096
GaussianNB() Tuned model TEST results on 2016-17:  0.5712927756653993 0.6232247284878863
LogisticRegr Tuned model TEST results on 2016-17:  0.6036121673003803 0.7188132164531355
BaggingClass Tuned model TEST results on 2016-17:  0.5456273764258555 0.6218354430379748
GradientBoos Tuned model TEST results on 2016-17:  0.5846007604562737 0.6867383512544804
XGBClassifie Tuned model TEST results on 2016-17:  0.5893536121673004 0.6795252225519289


In [26]:
x_17_sc = std_scal.transform(x_17)
x_test2 = x_17_sc.copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'NOT Tuned model TEST results on 2016-17: ', acc, f1)

SVC() NOT Tuned model TEST results on 2016-17:  0.5979087452471483 0.7128309572301427
RidgeClassif NOT Tuned model TEST results on 2016-17:  0.6074144486692015 0.7215104517869183
RandomForest NOT Tuned model TEST results on 2016-17:  0.5247148288973384 0.6081504702194357
GaussianNB() NOT Tuned model TEST results on 2016-17:  0.5712927756653993 0.6232247284878863
LogisticRegr NOT Tuned model TEST results on 2016-17:  0.6017110266159695 0.7163168584969533
BaggingClass NOT Tuned model TEST results on 2016-17:  0.49619771863117873 0.5250896057347669
GradientBoos NOT Tuned model TEST results on 2016-17:  0.5903041825095057 0.6928011404133999
XGBClassifie NOT Tuned model TEST results on 2016-17:  0.5532319391634981 0.6310832025117739


In [27]:
x_16_sc = std_scal.transform(x_16)
x_test2 = x_16_sc.copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc2, lrc2, rfc2, gnb2, lgr2, bc2, gbc2, xgbc2]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'NOT Tuned model TEST results on 20150-16 ', acc, f1, 'training : ', acct, f1t)

SVC() NOT Tuned model TEST results on 20150-16  0.5886792452830188 0.6872309899569583 training :  0.7216163060969069 0.7750975010833455
RidgeClassif NOT Tuned model TEST results on 20150-16  0.5877358490566038 0.6880799428979301 training :  0.7216163060969069 0.7750975010833455
RandomForest NOT Tuned model TEST results on 20150-16  0.539622641509434 0.6032520325203251 training :  0.7216163060969069 0.7750975010833455
GaussianNB() NOT Tuned model TEST results on 20150-16  0.5726415094339623 0.6050566695727986 training :  0.7216163060969069 0.7750975010833455
LogisticRegr NOT Tuned model TEST results on 20150-16  0.5886792452830188 0.6867816091954023 training :  0.7216163060969069 0.7750975010833455
BaggingClass NOT Tuned model TEST results on 20150-16  0.5481132075471699 0.5633546034639927 training :  0.7216163060969069 0.7750975010833455
GradientBoos NOT Tuned model TEST results on 20150-16  0.5830188679245283 0.6666666666666666 training :  0.7216163060969069 0.7750975010833455
XGBClas

In [28]:
x_16_sc = std_scal.transform(x_16)
x_test2 = x_16_sc.copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12],  'NOT Tuned model TEST results on 20150-16 ', acc, f1, 'training : ', acct, f1t)

SVC(C=10) NOT Tuned model TEST results on 20150-16  0.5820754716981132 0.6735445836403832 training :  0.7216163060969069 0.7750975010833455
RidgeClassif NOT Tuned model TEST results on 20150-16  0.5867924528301887 0.6866952789699571 training :  0.7216163060969069 0.7750975010833455
RandomForest NOT Tuned model TEST results on 20150-16  0.5433962264150943 0.6327769347496207 training :  0.7216163060969069 0.7750975010833455
GaussianNB() NOT Tuned model TEST results on 20150-16  0.5726415094339623 0.6050566695727986 training :  0.7216163060969069 0.7750975010833455
LogisticRegr NOT Tuned model TEST results on 20150-16  0.5877358490566038 0.6862885857860732 training :  0.7216163060969069 0.7750975010833455
BaggingClass NOT Tuned model TEST results on 20150-16  0.5377358490566038 0.5996732026143792 training :  0.7216163060969069 0.7750975010833455
GradientBoos NOT Tuned model TEST results on 20150-16  0.589622641509434 0.6746447270007478 training :  0.7216163060969069 0.7750975010833455
XGB

Now try regression models on goal differential ... converted to prediction

In [30]:
def make_win(x):
    if x>=0:
        return 1
    else:
        return 0
    
v_win = np.vectorize(make_win)

In [31]:
##regression models now ...

y = Y['goal_diff_target'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


lr = Ridge(alpha=50000) 
rfr = RandomForestRegressor(max_depth=4, random_state=0)
xgbr = XGBRegressor()


##quick checks 
for model in [lr, rfr, xgbr]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
    y_predt= model.predict(x_train_sc)  
    y_predwt = v_win(y_predt) 
    y_trainw = v_win(y_train)
    y_testw = v_win(y_test)  #same as usual win/loss
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    acct = accuracy_score(y_trainw, y_predwt)
    f1t = f1_score(y_trainw,y_predwt)
    
  
    print(str(model)[0:20], 'TEST: ', acc, f1 ,'training : ', acct, f1t)
    

[0.51690533 0.36376034 0.18694447 0.27135373 0.50487017]
Ridge(alpha=50000) TEST:  0.6197283774124375 0.7622877569258267 training :  0.619881995351332 0.7614452423698383
[0.85707395 0.43331879 0.02921838 0.14527683 0.87678157]
RandomForestRegresso TEST:  0.5782701929949964 0.6875 training :  0.6241730734847131 0.7214418234826397
[ 0.32998556  0.18154603 -0.6979711  -0.42923597 -0.45748696]
XGBRegressor(base_sc TEST:  0.5518227305218013 0.6369426751592356 training :  0.8975505095655283 0.9153493869109175


In [32]:
##regression models now ...
x_test = x_16.copy()
y_test = Y_16['goal_diff_target'].copy()

#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##quick checks 
for model in [lr, rfr, xgbr]:
   
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
   
    y_testw = v_win(y_test)  #same as usual win/loss
    
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    
  
    print(str(model)[0:12], 'TEST: ', acc, f1 )


[0.68702936 0.35889007 0.40138433 0.30298112 0.18451694]
Ridge(alpha= TEST:  0.5962264150943396 0.7440191387559809
[0.53925406 0.59556936 0.70076785 0.33123691 0.09648971]
RandomForest TEST:  0.6103773584905661 0.7109867039888034
[-0.4977314   0.12039401  1.1607935   0.45659927 -0.72441846]
XGBRegressor TEST:  0.5490566037735849 0.6294573643410853


In [33]:
##regression models now ...
x_test = x_17.copy()
y_test = Y_17['goal_diff_target'].copy()

#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##quick checks 
for model in [lr, rfr, xgbr]:
   
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
   
    y_testw = v_win(y_test)  #same as usual win/loss
    
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    
  
    print(str(model)[0:12], 'TEST: ', acc, f1 )

[0.1986591  0.15493199 0.40366176 0.50203642 0.08442206]
Ridge(alpha= TEST:  0.5998098859315589 0.7495538370017846
[-0.04304532 -0.10187141  0.3700241   0.56126893 -0.23675489]
RandomForest TEST:  0.6017110266159695 0.7104353835521771
[ 0.5456671   0.07267933 -0.39206013  1.918616    0.21196198]
XGBRegressor TEST:  0.55893536121673 0.6419753086419753
